In [ ]:
import spacy
import pandas as pd
from scispacy_fun.CustomSpacyF import  CustomSpacyF

## load sicspacy's model

In [ ]:
strVer = "10"
csf  = CustomSpacyF()


##  get single word's vector from scispacy model

In [ ]:
print(csf.getVector("psychosis"))

##  get sentence's vector from scispacy model


In [ ]:
print(csf.getSentenceVector("heart type disease"))
print(csf.getSentenceVector("neurological type disease"))

## load disease and icd code form config

In [ ]:
import toml 

dicTmp = toml.load("icd.toml")

dicFIcdTen = {}

strCodeList = ""
for disease ,icd in dicTmp.items():
    for version ,code in icd.items():
        if version == 'v'+strVer:
            strCodeList = "','".join(code)
            dicFIcdTen[disease] = "'"+strCodeList+"'"

## connect to mimic db

In [ ]:
import psycopg2

host = "203.64.95.185"
dbname = "postgres"
user = "nkust1108"
password = "nkust1108"

conn_string = f"host={host} dbname={dbname} user={user} password={password}"

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

def excuteSqlToDict(cursor,strSql):
    dic = {}

    strKey = ""
    strVal = ""

    cursor.execute(strSql)

    for row in cursor:
        strKey = row[0].strip()
        strVal = row[1].strip()
        dic[strKey] = strVal
    return dic


## get disease feature(long title) by icd code from mimic db

In [ ]:
def excuteSqlToDict(cursor,strSql):
    dic = {}

    strKey = ""
    strVal = ""

    cursor.execute(strSql)

    for row in cursor:
        strKey = row[0].strip()
        strVal = row[1].strip()
        dic[strKey] = strVal
    return dic


In [ ]:
def getDiseaseFeatureByIcdCode(dicFIcdTen,blnContainsBlank=False,strVer = "10"):
    strSql = ""
    dicIcdInfo = {}
    dicTmp = {}

    dicRtn={}
    for disease,strIcdCodeList in dicFIcdTen.items():

        strSql = """
            select
                icd_code,long_title 
            from 
                mimic_hosp.d_icd_diagnoses
            where
                icd_version = """+ strVer +"""
            and
                icd_code in (""" + strIcdCodeList + """)
            order by icd_code asc
        """

        dicIcdInfo = excuteSqlToDict(cursor,strSql)
        
        dicTmp = {}
        for code,title in dicIcdInfo.items():
            if blnContainsBlank == False:
                if  ((" ") in title) == False:
                    dicTmp[code] = title
            else:
                dicTmp[code] = title

        if len(dicTmp) != 0:         
            dicRtn[disease] = dicTmp

    return dicRtn

#get disease feature
dicIcdInfo = getDiseaseFeatureByIcdCode(dicFIcdTen,blnContainsBlank = True,strVer = strVer)

print(len(dicIcdInfo))
# for disease,dic in dicIcdInfo.items():
#     print(disease) 
#     for code,title in dic.items():
#         print(code,title)
#     print("")


## get disease feature's vector and export to csv

In [ ]:
dicVector = {}
dicTmp = {}
aryVec = []

for disease,dic in dicIcdInfo.items():

    dicTmp = {}
    for code,title in dic.items():
        aryVec = csf.getSentenceVector(title,blnStayComma = False,blnStayBrackets = False)
        #print(title,vec)
        strNewTitle = aryVec[0]
        odjVec = aryVec[1]
        dicTmp[code] = [strNewTitle,odjVec]

    dicVector[disease] = dicTmp
    

In [ ]:
lstDisease = []
lstIcdCode = []
lstTitle = []
lstVec = []
for disease,dic in dicVector.items():
    for code,ary in dic.items():
        lstDisease.append(disease)
        lstIcdCode.append(code)
        lstTitle.append(ary[0])
        lstVec.append(ary[1])
        print(ary[0],ary[1])

from datetime import datetime
strDate = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
strFileName = "icd"+strVer+"_vec_" + strDate + ".csv"
d = {"disease":lstDisease,"icd_code":lstIcdCode,"title":lstTitle,"vector":lstVec}
df = pd.DataFrame(data=d)
df.to_csv(strFileName)
